In [ ]:
from PIL import Image, ImageOps
import numpy as np
import os
import holoviews as hv; hv.extension('bokeh', logo=False)

hv.opts.defaults(hv.opts.Image(cmap="gray",invert_yaxis=True, xaxis=None, yaxis=None, data_aspect=1))

In [ ]:
def image_axes(im):
    inch_mm          = 25.4

    dpi              = im.info['dpi']
    xmax_mm, ymax_mm = im.width/dpi[0]*inch_mm,im.height/dpi[1]*inch_mm
    xs               = np.linspace(0,xmax_mm,im.width)
    ys               = np.linspace(0,ymax_mm,im.height)
    return xs, ys

In [ ]:
img  = Image.open( os.path.join('..','DATA','dog.jpg') )
A    = np.array(img)
B    = np.array(ImageOps.grayscale(img)) # Convert RGB to grayscale

img_x, img_y = image_axes( img )

In [ ]:
Bt = np.fft.fft2(B)
Btsort = np.sort(np.abs(Bt.reshape(-1))) # sort by magnitude

# Zero out all small coefficients and inverse transform
images = []
for keep in (0.1, 0.05, 0.01, 0.002):
    thresh  = Btsort[int(np.floor((1-keep)*len(Btsort)))]
    ind     = np.abs(Bt)>thresh          # Find small indices
    Atlow   = Bt * ind                   # Threshold small indices
    Alow    = np.fft.ifft2(Atlow).real   # Compressed image
    images.append(  hv.Image((img_x,img_y, Alow ), kdims=['x','y'], vdims="data").opts(title=f'keep={keep:.2}') )
hv.Layout(images).opts(shared_axes=False, title='Compressed Image')